## Function to load, concatenate and calculate Max Daily Temp

In [1]:
# Libraries
%load_ext autoreload
%autoreload


import numpy as np
import iris
import iris.coord_categorisation
from iris.util import unify_time_units
from iris.experimental.equalise_cubes import equalise_attributes
import iris.quickplot as qplt
# import pandas as pd

import glob
import sys
sys.path.append('../')
from libs.plot_maps import *


import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy.ma as ma
%matplotlib inline
import cartopy.crs as ccrs

ImportError: 
Importing the multiarray numpy extension module failed.  Most
likely you are trying to import a failed build of numpy.
If you're working with a numpy git repo, try `git clean -xdf` (removes all
files not under version control).  Otherwise reinstall numpy.

Original error was: libopenblas.so.0: cannot open shared object file: No such file or directory


In [ ]:
path     = '../data/aus_wind/'
path_out = '../data/aus_wind/'
outfile = 'temp.max.'

years = range(2000, 2002)

In [ ]:
def format_climate(years, path, path_out, outfile, cal_wind=False, file_name='air.2m.gauss.'):
    '''Function to condense multiple years of x4 daily measurements into maximum daily, monthly-averages.
    Produces one netcdf file at the end.
    
    Variables are:
    + years = the year ranges you want to condense e.g range(2000, 2002) will be 2000, 2001
    
    + path = directory with *all* data
    
    + path_out = the exit directory where you want the data saved
    
    + outfile = the output file name. The year range and .nc will be added at the end of this
    
    + cal_wind = Boolean argument
                If True, the function will assume you're directing it to path with both v and u files.
                It will calculate total wind, s, as described in `cal_wnd_spd.ipynb`.
                
                If False, the function will load temperature data and only calcualte max daily, monthly averages.
    
    + file_name = a string with the main section of the file name. e.g. for 'air.2m.gauss.2011.nc', this would be
                'air.2m.gauss.' You don't need this if you are calculating wind speed as the file names are already
                in the function :)'''

    if cal_wind:
        u_files = []
        v_files = []
        for year in years:
            u_files.append(path + 'uwnd.10m.gauss.' + str(year) + '.nc')
            v_files.append(path + 'vwnd.10m.gauss.' + str(year) + '.nc')

        uList = []
        vList = []

        # Loading in files
        print('Loading in u files...')
        for i in u_files:
            dfu = iris.load_cube(i)
            dfu.coord('time').attributes = {}
            uList.append(dfu)
        print('Files loaded.')

        print('Loading in v files...')
        for j in v_files:
            dfv = iris.load_cube(j)
            dfv.coord('time').attributes = {}
            vList.append(dfv)
        print('Files loaded.')

        ### For all u and v files
        # u_files = glob.glob(path + 'u*.nc') # use 'recursive = True' to include subdirectories
        # u_files = sorted(u_files) # sort alphabetically
        # v_files = glob.glob(path + 'v*.nc') 

        u_cube_list = iris.cube.CubeList(uList)
        v_cube_list = iris.cube.CubeList(vList)
        slist = v_cube_list.copy()
        
        equalise_attributes(u_cube_list)
        unify_time_units(u_cube_list)

        equalise_attributes(v_cube_list)
        unify_time_units(v_cube_list)

        print('Calculating wind speed...')
        for year in range(len(years)):
            v_dat = v_cube_list[year].data
            u_dat = u_cube_list[year].data
            s_dat = np.sqrt((v_dat * v_dat) + (u_dat * u_dat))

            slist[year].data = s_dat
        print('Wind speed calculated.')
        t_cube_list = iris.cube.CubeList(slist)

    
    else:
        t_files = []
        for year in years:
            t_files.append(path + file_name + str(year) + '.nc')

        tList = []
        print('Loading in files...')
        for i in t_files:
            df = iris.load_cube(i)
            df.coord('time').attributes = {}
            tList.append(df)
        print('Files loaded.')

        # Turn into cube list
        t_cube_list = iris.cube.CubeList(tList)

        # So all units match up
        equalise_attributes(t_cube_list)
        unify_time_units(t_cube_list)

    # Concatenate cubes together (end of if loop)
    print('Concatenating...')
    t = t_cube_list.concatenate_cube()
    print('Concatenating complete')
    
    # Add coordinates in
    print('Adding coordinates...')
    try:
        iris.coord_categorisation.add_month(t, 'time', name='month')
        iris.coord_categorisation.add_day_of_year(t, 'time', name='day_of_year')
        iris.coord_categorisation.add_year(t, 'time', name='year')
        print('Success.')
    except:
        pass
    
    # Max daily values
    test = s.aggregated_by(['day_of_year', 'year'], iris.analysis.MAX)
    swnd = test.aggregated_by(['month', 'year'], iris.analysis.MEAN)
    
    # Saving outputs
    print('Saving files...')
    out = path_out + outfile + str(years[0]) + '-' + str(years[len(years) -1]) + '.nc'
    print(out)
    iris.save(swnd, out)
    print('Save complete.')

In [ ]:
format_climate(cal_wind=False, years=years, path=path, path_out=path_out, outfile=outfile)